In [ ]:
import ast
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_recommenders as tfrs
from sklearn.model_selection import train_test_split


In [ ]:
raw_user_data = pd.read_json('../results/user_dataset.json')
raw_food_data = pd.read_json('../results/output.json')
listUserPreferences = pd.DataFrame(
    [data for data in raw_user_data.listUserPreferences])
listFoodPreferences = pd.DataFrame(
    [data for data in raw_food_data.food])
temp = pd.DataFrame(
    [data for data in listFoodPreferences.servings])
listFoodServings = pd.DataFrame(
    [item for sublist in temp.serving for item in sublist])
listFoodPreferences = listFoodPreferences.drop(columns=['servings'])
raw_listFood = listFoodPreferences.merge(
    listFoodServings, how='left', left_index=True, right_index=True)
listFood = raw_listFood.fillna(0)


In [ ]:
listUserPreferences


,gender,weight,id_user,activityLevel,goals,height,disease,favoriteFood,birthdate
0,Male,50,3222915,1.9,0,180,[],"[Chicken Cordon Bleu, Chocolate Mousse Cake, S...",20-10-1990
1,Male,46,9930952,1.725,0,151,[obesity],"[Samosa, Pizza Dough, Baguette, Chocolate Mous...",10-09-1976
2,Male,60,3717984,1.725,0,164,"[heart, hypertension]","[Bubur Ayam, Mashed Potato, Puff Pastry, Chick...",01-05-2019
3,Male,41,6726690,1.55,0,156,"[diabetes, heart, obesity]","[Fish Cake, Red Velvet, Black Forest, Guacamol...",02-06-1979
4,Female,66,6452326,1.2,-500,151,[],"[Puff Pastry, Mashed Potato, Bubur Ayam, Okono...",22-03-1988
...,...,...,...,...,...,...,...,...,...
260,Male,65,7559844,1.55,500,163,[],"[Chicken Cordon Bleu, Pad Thai, Churros, Black...",26-01-1975
261,Male,84,8260717,1.9,500,196,[heart],"[Bibimbap, Puff Pastry, Thai Tea, Churros, Cin...",14-05-1950
262,Female,47,4895731,1.55,-500,180,"[diabetes, heart]","[Chocolate Lava Cake, Meringue Cookies, Takoya...",21-07-2016
263,Male,63,7540747,1.55,500,183,"[diabetes, hypertension, obesity]","[Thai Tea, Chocolate Mousse Cake, Pad Thai, Me...",22-10-1952


In [ ]:
listFood


,food_id,food_name,food_type,food_url,brand_name,calcium,calories,carbohydrate,cholesterol,fat,...,serving_description,serving_id,serving_url,sodium,sugar,trans_fat,vitamin_a,vitamin_c,added_sugars,vitamin_d
0,2655309,Kebab,[],https://www.fatsecret.com/calories-nutrition/g...,0,176,620,77.10,53,16.63,...,1 kebab,2590791,https://www.fatsecret.com/calories-nutrition/g...,1029,6.91,0,37,22.2,0,0
1,58704864,Gyoza,[],https://www.fatsecret.com/calories-nutrition/h...,Hungry Planet,176,620,77.10,53,16.63,...,1 serving (390 g),2590792,https://www.fatsecret.com/calories-nutrition/g...,1029,6.91,0,37,22.2,0,0
2,68848666,Gyoza,[],https://www.fatsecret.com/calories-nutrition/y...,Yoshinoya,45,159,19.77,13,4.27,...,100 g,2590793,https://www.fatsecret.com/calories-nutrition/g...,264,1.77,0,9,5.7,0,0
3,515347,Gyoza,[],https://www.fatsecret.com/calories-nutrition/t...,Tokyo Joe's,13,45,5.60,4,1.21,...,1 oz,2590794,https://www.fatsecret.com/calories-nutrition/g...,75,0.50,0,3,1.6,0,0
4,1684073,Chicken Katsu,[],https://www.fatsecret.com/calories-nutrition/l...,L&L Hawaiian Barbecue,30,120,17.00,0,2.50,...,4 gyoza,48898797,https://www.fatsecret.com/calories-nutrition/h...,280,1.00,0,0,0,1.00,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260,447317,Guacamole,[],https://www.fatsecret.com/calories-nutrition/c...,Chevy's Fresh Mex,0,330,41.00,0,15.00,...,1 serving,56535626,https://www.fatsecret.com/calories-nutrition/r...,390,6.00,0.500,0,0,0,0
261,2125933,Guacamole,[],https://www.fatsecret.com/calories-nutrition/c...,Costa Vida,0,100,12.00,0,4.50,...,1 croissant,44521664,https://www.fatsecret.com/calories-nutrition/p...,210,3.00,0,0,0,0,0
262,1995633,Guacamole,[],https://www.fatsecret.com/calories-nutrition/o...,Ortega,0,264,28.90,0,13.80,...,1 serving,56209720,https://www.fatsecret.com/calories-nutrition/t...,246,3.90,0.100,0,0,0,0
263,99383,Guacamole,[],https://www.fatsecret.com/calories-nutrition/c...,Chili's,0,250,25.00,35,14.00,...,1 serving,4438697,https://www.fatsecret.com/calories-nutrition/s...,230,3.00,0,0,0,0,0


In [ ]:
# Convert the list columns to strings
listFood['food_name'] = listFood['food_name'].astype(str)
# listUserPreferences['favoriteFood'] = listUserPreferences['favoriteFood'].astype(str).apply(eval).apply(lambda x: ', '.join(x))
listUserPreferences = listUserPreferences.explode('favoriteFood')
listFood['food_name']


0              Kebab
1              Gyoza
2              Gyoza
3              Gyoza
4      Chicken Katsu
           ...      
260        Guacamole
261        Guacamole
262        Guacamole
263        Guacamole
264        Guacamole
Name: food_name, Length: 265, dtype: object

In [ ]:

listUserPreferences['favoriteFood']


0        Chicken Cordon Bleu
0      Chocolate Mousse Cake
0                     Samosa
0               Cream Cheese
0                Carrot Cake
               ...          
264          Salmon Teriyaki
264               Burger Bun
264                 Pad Thai
264              Onion Rings
264                 Takoyaki
Name: favoriteFood, Length: 1981, dtype: object